## Import Libs

In [30]:
import pickle
import pandas as pd
import numpy as np
import joblib
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, auc, precision_recall_curve, roc_curve
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from plot_metric.functions import BinaryClassification
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [31]:
# check the system font
import matplotlib.font_manager as font_manager
font_manager.findSystemFonts(fontpaths=None, fontext='ttf')

# add the font wanted
font_dir = ['../../Latin-Modern-Roman']
for font in font_manager.findSystemFonts(font_dir):
    font_manager.fontManager.addfont(font)

# Set font family globally
plt.rcParams['font.family'] = 'Latin Modern Roman'
print(plt.rcParams['font.family'])

['Latin Modern Roman']


## Feature Selection 

In [32]:
def gather_dfs(data_path, patient_ids):

    gathered_df = pd.DataFrame([])

    for p in tqdm(patient_ids):
        # read in patient data
        p = str(np.char.replace(p,'psv','csv'))
        df = pd.read_csv(data_path + p, sep = ",")
        if 'time' in df.columns: df = df.drop(['time'],axis=1)   # drop column time if exists
        gathered_df = pd.concat([gathered_df, df], ignore_index=True)
    
    return gathered_df


In [33]:
def expand_sepsis_label(df):

    number_nonsepsis_label = df.groupby('sepsis')['sepsis'].count()[0]
    print('Number of Non-sepsis label:', number_nonsepsis_label)
    number_sepsis_label = df.groupby('sepsis')['sepsis'].count()[1]
    print('Number of Sepsis label:', number_sepsis_label)

    expand_times = round((number_nonsepsis_label-number_sepsis_label)/number_sepsis_label)

    df_is_sepsis = df['sepsis']==1
    df_to_expand = df[df_is_sepsis]
    df_expanded = df.append([df_to_expand]*expand_times, ignore_index = True)

    number_nonsepsis_label = df_expanded.groupby('sepsis')['sepsis'].count()[0]
    print('After Expansion:\nNumber of Non-sepsis label:', number_nonsepsis_label)
    number_sepsis_label = df_expanded.groupby('sepsis')['sepsis'].count()[1]
    print('Number of Sepsis label:', number_sepsis_label)

    return df_expanded


In [5]:
test_set_c = np.load('../data/data_Cinc2019/test_set.npy')
train_nonsepsis_c = np.load('../data/data_Cinc2019/train_nonsepsis.npy')
train_sepsis_c = np.load('../data/data_Cinc2019/train_sepsis.npy')
print(test_set_c.shape)
print(train_nonsepsis_c.shape)
print(train_sepsis_c.shape)

(880,)
(36964,)
(2492,)


In [6]:
test_set_m = np.load('../data/data_mimiciii/test_set.npy')
train_nonsepsis_m = np.load('../data/data_mimiciii/train_nonsepsis.npy')
train_sepsis_m = np.load('../data/data_mimiciii/train_sepsis.npy')
print(test_set_m.shape)
print(train_nonsepsis_m.shape)
print(train_sepsis_m.shape)

(566,)
(3097,)
(1603,)


### Baseline Data

In [7]:
features_baseline = pd.read_csv('../../datasets/Cinc2019/baseline_all/p000002.csv').columns
features_baseline

Index(['HR', 'SaO2', 'Temp', 'SBP', 'MAP', 'DBP', 'RR', 'BaseExcess', 'HCO3',
       'PH', 'BUN', 'Calcium', 'Chloride', 'Creatinine', 'Glucose', 'Lactic',
       'Magnesium', 'Potassium', 'PTT', 'WBC', 'Platelet', 'age', 'gender',
       'sepsis', 'subject_id'],
      dtype='object')

In [34]:
X_feature_baseline = ['HR', 'SaO2', 'Temp', 'SBP', 'MAP', 'DBP', 'RR', 'BaseExcess', 'HCO3',
       'PH', 'BUN', 'Calcium', 'Chloride', 'Creatinine', 'Glucose', 'Lactic',
       'Magnesium', 'Potassium', 'PTT', 'WBC', 'Platelet', 'age', 'gender']
y_feature = ['sepsis']

In [10]:
!mkdir -p ../data/data_both/kFold_baseline/fold1
!mkdir -p ../data/data_both/kFold_baseline/fold2
!mkdir -p ../data/data_both/kFold_baseline/fold3
!mkdir -p ../data/data_both/kFold_baseline/fold4
!mkdir -p ../data/data_both/kFold_baseline/fold5
!mkdir -p ./figs

In [11]:
# 5-fold cross validation was implemented to select the most important features
# meanwhile, 5 fold data will be saved for other models
sss = StratifiedShuffleSplit(n_splits=5, test_size = 3/17, random_state=np.random.seed(12306)) # val/train = 0.15/0.7
for (k, (train0_index, val0_index)), (k, (train1_index, val1_index)), (k, (train2_index, val2_index)), (k, (train3_index, val3_index))\
     in zip(enumerate(sss.split(train_nonsepsis_c, np.zeros(train_nonsepsis_c.shape))), enumerate(sss.split(train_sepsis_c, np.zeros(train_sepsis_c.shape))), enumerate(sss.split(train_nonsepsis_m, np.zeros(train_nonsepsis_m.shape))), enumerate(sss.split(train_sepsis_m, np.zeros(train_sepsis_m.shape)))):

    print('---Fold{}/5---'.format(k+1))
    
    # read data from Cinc2019
    train_set_c = np.append((train_nonsepsis_c[train0_index])[:2052], (train_sepsis_c[train1_index])[:2052])
    val_set_c = np.append((train_nonsepsis_c[val0_index])[:440], (train_sepsis_c[val1_index][:440]))
    train_baseline_c = gather_dfs('../../datasets/Cinc2019/baseline_all/',train_set_c)
    val_baseline_c = gather_dfs('../../datasets/Cinc2019/baseline_all/',val_set_c)

    # read data from MIMIC-III 
    train_set_m = np.append((train_nonsepsis_m[train2_index][:1320]), (train_sepsis_m[train3_index])[:1320])
    val_set_m = np.append((train_nonsepsis_m[val2_index])[:283], (train_sepsis_m[val3_index])[:283])
    train_baseline_m = gather_dfs('../../datasets/MIMICIII/adults/baseline_all/',train_set_m)
    val_baseline_m = gather_dfs('../../datasets/MIMICIII/adults/baseline_all/',val_set_m)

    # combine two datasets
    train_baseline_both = pd.concat([train_baseline_c,train_baseline_m], ignore_index=True)
    train_baseline = expand_sepsis_label(train_baseline_both) #balance sepsis and non-sepsis labels
    val_baseline_both = pd.concat([val_baseline_c,val_baseline_m], ignore_index=True)
    val_baseline = expand_sepsis_label(val_baseline_both) #balance sepsis and non-sepsis labels

    filename = '../data/data_both/kFold_baseline/fold{}/train.pickle'.format(k+1)
    with open(filename, "wb") as f:
        pickle.dump(train_baseline, f)
    filename = '../data/data_both/kFold_baseline/fold{}/val.pickle'.format(k+1)
    with open(filename, "wb") as f:
        pickle.dump(val_baseline, f)

    X_train_baseline = train_baseline[X_feature_baseline]
    y_train_baseline = train_baseline[y_feature]

    # visualize the feature importance
    plt.figure(figsize=(20,10))
    model = RandomForestClassifier(n_estimators=100, random_state=1211)
    model.fit(X_train_baseline,y_train_baseline)
    #plot graph of feature importances for better visualization
    feat_importances = pd.Series(model.feature_importances_, index=X_train_baseline.columns)
    feat_importances.nlargest(len(X_feature_baseline)).plot(kind='barh')
    plt.savefig('./figs/feature_importance_baseline_{}.pdf'.format(k+1))
    plt.clf()

    # select the features based on importance while using the RandomForestClassifier
    rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100,random_state=1211), max_features=len(X_feature_baseline))
    rf_selector.fit(X_train_baseline, y_train_baseline)

    # use get_support() to get the features
    rf_support = rf_selector.get_support()
    rf_feature_baseline = X_train_baseline.loc[:,rf_support].columns.tolist()
    print(str(len(rf_feature_baseline)), 'selected features for baseline data')
    print('Selected Features: ',rf_feature_baseline)

---Fold1/5---


100%|██████████| 566/566 [00:03<00:00, 143.23it/s]


Number of Non-sepsis label: 346863
Number of Sepsis label: 28770
After Expansion:
Number of Non-sepsis label: 346863
Number of Sepsis label: 345240
Number of Non-sepsis label: 72765
Number of Sepsis label: 6174
After Expansion:
Number of Non-sepsis label: 72765
Number of Sepsis label: 74088


/Users/jinyijin/.conda/envs/DL_Environment/lib/python3.7/site-packages/ipykernel_launcher.py:40: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/jinyijin/.conda/envs/DL_Environment/lib/python3.7/site-packages/sklearn/feature_selection/_from_model.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)


11 selected features for baseline data
Selected Features:  ['HR', 'Temp', 'BUN', 'Calcium', 'Creatinine', 'Glucose', 'Potassium', 'PTT', 'WBC', 'Platelet', 'age']
---Fold2/5---


...

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

Results for feature selection

In [35]:
# 10 features from the feature selection + 5 extra vital signs = 15 features
X_feature_baseline_fs = ['HR', 'Temp', 'BUN', 'Calcium', 'Creatinine', 'Glucose', 'PTT', 'WBC', 'Platelet', 'age',
                         'RR', 'SaO2','MAP','SBP', 'DBP']

### Engineered Data

In [10]:
features_eng = pd.read_csv('../../datasets/Cinc2019/engineered_all/p000002.csv').columns
features_eng

Index(['HR', 'SaO2', 'Temp', 'SBP', 'MAP', 'DBP', 'RR', 'BaseExcess', 'HCO3',
       'PH', 'BUN', 'Calcium', 'Chloride', 'Creatinine', 'Glucose', 'Lactic',
       'Magnesium', 'Potassium', 'PTT', 'WBC', 'Platelet', 'age', 'gender',
       'sepsis', 'subject_id', 'HR_dev_1', 'HR_dev_2', 'HR_dev_3', 'RR_dev_1',
       'RR_dev_2', 'RR_dev_3', 'Temp_dev_1', 'Temp_dev_2', 'Temp_dev_3',
       'Bradycardia', 'Tachycardia', 'Hypothermia', 'Fever', 'Hyperpyrexia'],
      dtype='object')

In [36]:
X_feature_engineered = ['HR', 'SaO2', 'Temp', 'SBP', 'MAP', 'DBP', 'RR', 'BaseExcess', 'HCO3',
       'PH', 'BUN', 'Calcium', 'Chloride', 'Creatinine', 'Glucose', 'Lactic',
       'Magnesium', 'Potassium', 'PTT', 'WBC', 'Platelet', 'age', 'gender',
       'HR_dev_1', 'HR_dev_2', 'HR_dev_3', 'RR_dev_1',
       'RR_dev_2', 'RR_dev_3', 'Temp_dev_1', 'Temp_dev_2', 'Temp_dev_3',
       'Bradycardia', 'Tachycardia', 'Hypothermia', 'Fever', 'Hyperpyrexia']
y_feature = ['sepsis']

In [14]:
!mkdir -p ../data/data_both/kFold_engineered/fold1
!mkdir -p ../data/data_both/kFold_engineered/fold2
!mkdir -p ../data/data_both/kFold_engineered/fold3
!mkdir -p ../data/data_both/kFold_engineered/fold4
!mkdir -p ../data/data_both/kFold_engineered/fold5

In [15]:
# 5-fold cross validation was implemented to select the most important features
# meanwhile, 5 fold data will be saved for other models
sss = StratifiedShuffleSplit(n_splits=5, test_size = 3/17, random_state=np.random.seed(12306)) # val/train = 0.15/0.7
for (k, (train0_index, val0_index)), (k, (train1_index, val1_index)), (k, (train2_index, val2_index)), (k, (train3_index, val3_index))\
     in zip(enumerate(sss.split(train_nonsepsis_c, np.zeros(train_nonsepsis_c.shape))), enumerate(sss.split(train_sepsis_c, np.zeros(train_sepsis_c.shape))), enumerate(sss.split(train_nonsepsis_m, np.zeros(train_nonsepsis_m.shape))), enumerate(sss.split(train_sepsis_m, np.zeros(train_sepsis_m.shape)))):

    print('---Fold{}/5---'.format(k+1))
    
    # read data from Cinc2019
    train_set_c = np.append((train_nonsepsis_c[train0_index])[:2052], (train_sepsis_c[train1_index])[:2052])
    val_set_c = np.append((train_nonsepsis_c[val0_index])[:440], (train_sepsis_c[val1_index][:440]))
    train_engineered_c = gather_dfs('../../datasets/Cinc2019/engineered_all/',train_set_c)
    val_engineered_c = gather_dfs('../../datasets/Cinc2019/engineered_all/',val_set_c)

    # read data from MIMIC-III 
    train_set_m = np.append((train_nonsepsis_m[train2_index][:1320]), (train_sepsis_m[train3_index])[:1320])
    val_set_m = np.append((train_nonsepsis_m[val2_index])[:283], (train_sepsis_m[val3_index])[:283])
    train_engineered_m = gather_dfs('../../datasets/MIMICIII/adults/engineered_all/',train_set_m)
    val_engineered_m = gather_dfs('../../datasets/MIMICIII/adults/engineered_all/',val_set_m)

    # combine two datasets
    train_engineered_both = pd.concat([train_engineered_c,train_engineered_m], ignore_index=True)
    train_engineered = expand_sepsis_label(train_engineered_both) #balance sepsis and non-sepsis labels
    val_engineered_both = pd.concat([val_engineered_c,val_engineered_m], ignore_index=True)
    val_engineered = expand_sepsis_label(val_engineered_both) #balance sepsis and non-sepsis labels

    filename = '../data/data_both/kFold_engineered/fold{}/train.pickle'.format(k+1)
    with open(filename, "wb") as f:
        pickle.dump(train_engineered, f)
    filename = '../data/data_both/kFold_engineered/fold{}/val.pickle'.format(k+1)
    with open(filename, "wb") as f:
        pickle.dump(val_engineered, f)

    X_train_engineered = train_engineered[X_feature_engineered]
    y_train_engineered = train_engineered[y_feature]

    # visualize the feature importance
    plt.figure(figsize=(20,10))
    model = RandomForestClassifier(n_estimators=100, random_state=1211)
    model.fit(X_train_engineered,y_train_engineered)
    #plot graph of feature importances for better visualization
    feat_importances = pd.Series(model.feature_importances_, index=X_train_engineered.columns)
    feat_importances.nlargest(len(X_feature_engineered)).plot(kind='barh')
    plt.savefig('./figs/feature_importance_engineered_{}.pdf'.format(k+1))
    plt.clf()

    # select the features based on importance while using the RandomForestClassifier
    embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100,random_state=1211), max_features=len(X_feature_engineered))
    embeded_rf_selector.fit(X_train_engineered, y_train_engineered)

    # use get_support() to get the features
    embeded_rf_support = embeded_rf_selector.get_support()
    embeded_rf_feature_engineered = X_train_engineered.loc[:,embeded_rf_support].columns.tolist()
    print(str(len(embeded_rf_feature_engineered)), 'selected features for engineered data')
    print(embeded_rf_feature_engineered)

---Fold1/5---


100%|██████████| 566/566 [00:04<00:00, 121.05it/s]


Number of Non-sepsis label: 346863
Number of Sepsis label: 28770
After Expansion:
Number of Non-sepsis label: 346863
Number of Sepsis label: 345240
Number of Non-sepsis label: 72765
Number of Sepsis label: 6174
After Expansion:
Number of Non-sepsis label: 72765
Number of Sepsis label: 74088


/Users/jinyijin/.conda/envs/DL_Environment/lib/python3.7/site-packages/ipykernel_launcher.py:40: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/jinyijin/.conda/envs/DL_Environment/lib/python3.7/site-packages/sklearn/feature_selection/_from_model.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)


15 selected features for engineered data
['Temp', 'BUN', 'Calcium', 'WBC', 'Platelet', 'age', 'HR_dev_1', 'HR_dev_2', 'HR_dev_3', 'RR_dev_1', 'RR_dev_2', 'RR_dev_3', 'Temp_dev_1', 'Temp_dev_2', 'Temp_dev_3']
---Fold2/5---


...

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

Results for feature selection

In [37]:
# 21 features from feature selection + 5 extra vital signs = 26 features
X_feature_engineered_fs = ['HR', 'Temp', 'PH', 'BUN', 'Calcium', 'Creatinine', 'Glucose', 
                            'Lactic', 'PTT', 'WBC', 'Platelet', 'age', 'HR_dev_1', 'HR_dev_2', 
                            'HR_dev_3', 'RR_dev_1', 'RR_dev_2', 'RR_dev_3', 'Temp_dev_1', 'Temp_dev_2', 'Temp_dev_3',
                            'RR', 'SBP', 'DBP','MAP', 'SaO2']

## Hyper-parameter Tuning

### Baseline Data W/o Feature Engineering

In [ ]:
for k in tqdm(range(5)):
    print('---Fold{}/5---'.format(k+1))
    
    # load the train and val data
    filename = '../data/data_both/kFold_baseline/fold{}/train.pickle'.format(k+1)
    with open(filename, 'rb') as f:
        train_baseline = pickle.load(f)

    filename = '../data/data_both/kFold_baseline/fold{}/val.pickle'.format(k+1)
    with open(filename, 'rb') as f:
        val_baseline = pickle.load(f)
    
    X_train_baseline = train_baseline[X_feature_baseline]
    y_train_baseline = train_baseline[y_feature]

    X_val_baseline = val_baseline[X_feature_baseline]
    y_val_baseline = val_baseline[y_feature]

    # params for searching
    param_grid = {
        'n_estimators':[50, 100, 150], 
        'max_depth':[10, 20, 30]} #param_grid
    rf = GridSearchCV(RandomForestClassifier(random_state=1211, oob_score=True, class_weight='balanced_subsample'), param_grid, verbose = 2 ,n_jobs = -1, scoring='accuracy')
    rf.fit(X_train_baseline,y_train_baseline.values.ravel())
    print(rf.best_params_)

Results:

 {'max_depth': 30, 'n_estimators': 150}

### Baseline Data W Feature Engineering

In [ ]:
for k in tqdm(range(5)):
    print('---Fold{}/5---'.format(k+1))
    
    # load the train and val data
    filename = '../data/data_both/kFold_baseline/fold{}/train.pickle'.format(k+1)
    with open(filename, 'rb') as f:
        train_baseline = pickle.load(f)

    filename = '../data/data_both/kFold_baseline/fold{}/val.pickle'.format(k+1)
    with open(filename, 'rb') as f:
        val_baseline = pickle.load(f)
    
    X_train_baseline = train_baseline[X_feature_baseline_fs]
    y_train_baseline = train_baseline[y_feature]

    X_val_baseline = val_baseline[X_feature_baseline_fs]
    y_val_baseline = val_baseline[y_feature]

    # params for searching
    param_grid = {
        'n_estimators':[50, 100, 150], 
        'max_depth':[10, 20, 30]} #param_grid
    rf = GridSearchCV(RandomForestClassifier(random_state=1211, oob_score=True, class_weight='balanced_subsample'), param_grid, verbose = 2 ,n_jobs = -1, scoring='accuracy')
    rf.fit(X_train_baseline,y_train_baseline.values.ravel())
    print(rf.best_params_)

Results:

 {'max_depth': 30, 'n_estimators': 150}

### Engineered Data W/o Feature Engineering

In [ ]:
for k in tqdm(range(5)):
    print('---Fold{}/5---'.format(k+1))
    
    # load the train and val data
    filename = '../data/data_both/kFold_engineered/fold{}/train.pickle'.format(k+1)
    with open(filename, 'rb') as f:
        train_engineered = pickle.load(f)

    filename = '../data/data_both/kFold_engineered/fold{}/val.pickle'.format(k+1)
    with open(filename, 'rb') as f:
        val_engineered = pickle.load(f)
    
    X_train_engineered = train_engineered[X_feature_engineered]
    y_train_engineered = train_engineered[y_feature]

    X_val_engineered = val_engineered[X_feature_engineered]
    y_val_engineered = val_engineered[y_feature]

    # params for searching
    param_grid = {
        'n_estimators':[50, 100, 150], 
        'max_depth':[10, 20, 30]} #param_grid
    rf = GridSearchCV(RandomForestClassifier(random_state=1211, oob_score=True, class_weight='balanced_subsample'), param_grid, verbose = 2 ,n_jobs = -1, scoring='accuracy')
    rf.fit(X_train_engineered,y_train_engineered.values.ravel())
    print(rf.best_params_)

Results:

{'max_depth': 30, 'n_estimators': 50}

### Engineered Data W Feature Engineering

In [ ]:
for k in tqdm(range(5)):
    print('---Fold{}/5---'.format(k+1))
    
    # load the train and val data
    filename = '../data/data_both/kFold_engineered/fold{}/train.pickle'.format(k+1)
    with open(filename, 'rb') as f:
        train_engineered = pickle.load(f)

    filename = '../data/data_both/kFold_engineered/fold{}/val.pickle'.format(k+1)
    with open(filename, 'rb') as f:
        val_engineered = pickle.load(f)
    
    X_train_engineered = train_engineered[X_feature_engineered_fs]
    y_train_engineered = train_engineered[y_feature]

    X_val_engineered = val_engineered[X_feature_engineered_fs]
    y_val_engineered = val_engineered[y_feature]

    # params for searching
    param_grid = {
        'n_estimators':[50, 100, 150], 
        'max_depth':[10, 20, 30]} #param_grid
    rf = GridSearchCV(RandomForestClassifier(random_state=1211, oob_score=True, class_weight='balanced_subsample'), param_grid, verbose = 2 ,n_jobs = -1, scoring='accuracy')
    rf.fit(X_train_engineered,y_train_engineered.values.ravel())
    print(rf.best_params_)

Results:

 {'max_depth': 30, 'n_estimators': 150}

## Train Models Using The Best Params

In [17]:
!mkdir -p ./trained_models/BDWOFS
!mkdir -p ./trained_models/BDWFS
!mkdir -p ./trained_models/EDWOFS
!mkdir -p ./trained_models/EDWFS

### Baseline Data W/o Feature Selection

In [19]:
f1_list = []
accuracy_list = []
auprc_list = []
auroc_list = []

for k in range(5):
    print('---Fold{}/5---'.format(k+1))
    
    # load the train and val data
    filename = '../data/data_both/kFold_baseline/fold{}/train.pickle'.format(k+1)
    with open(filename, 'rb') as f:
        train_baseline = pickle.load(f)

    filename = '../data/data_both/kFold_baseline/fold{}/val.pickle'.format(k+1)
    with open(filename, 'rb') as f:
        val_baseline = pickle.load(f)
    
    X_train_baseline = train_baseline[X_feature_baseline]
    y_train_baseline = train_baseline[y_feature]

    X_val_baseline = val_baseline[X_feature_baseline]
    y_val_baseline = val_baseline[y_feature]

    # train and save the models
    best_RF = RandomForestClassifier(random_state=1211, oob_score=True, max_depth=30, n_estimators=150, class_weight='balanced_subsample')
    best_RF.fit(X_train_baseline,y_train_baseline.values.ravel())
    joblib.dump(best_RF, './trained_models/BDWOFS/model{}.joblib'.format(k+1))

    # train results
    y_pred_class = best_RF.predict(X_val_baseline)
    y_pred_proba = best_RF.predict_proba(X_val_baseline)[::,1]
    precision_, recall_, thresholds_ = precision_recall_curve(y_val_baseline, y_pred_proba)
    auprc_ = auc(recall_, precision_)
    fpr_, tpr_, thresholds_ = roc_curve(y_val_baseline, y_pred_proba)
    auroc_ = auc(fpr_, tpr_)
    f1_ = f1_score(y_val_baseline,y_pred_class)
    accuracy_ = accuracy_score(y_val_baseline,y_pred_class)

    print('Classification Report:\n', classification_report(y_val_baseline, y_pred_class))
    print('F1 score:', f1_)
    print('Accuracy:', accuracy_)
    print('AUPRC:', auprc_)
    print('AUROC:', auroc_)
    print('AUROC threshold:', thresholds_)

    f1_list.append(f1_)
    accuracy_list.append(accuracy_)
    auprc_list.append(auprc_)
    auroc_list.append(auroc_)

    # Plot ROC,PRC and confusion matrix
    bc = BinaryClassification(y_val_baseline,y_pred_proba, labels=['Non-sepsis', 'Sepsis'])
    plt.figure(figsize=(20,20))
    plt.subplot2grid(shape=(2,4), loc=(0,0), colspan=2)
    bc.plot_roc_curve()
    plt.subplot2grid((2,4), (0,2), colspan=2)
    bc.plot_precision_recall_curve()
    plt.subplot2grid((2,4), (1,0), colspan=2)
    bc.plot_confusion_matrix()
    plt.subplot2grid((2,4), (1,2), colspan=2)
    bc.plot_confusion_matrix(normalize=True)
    plt.savefig('./figs/val_wo_fs_baseline_{}.pdf'.format(k+1))
    plt.clf()

def Average(lst):
    return sum(lst) / len(lst)

print('Averaged trauining results:\n F1_score:{}\n Accuracy:{}\n AUROC:{}\n AUPRC:{}\n'.format(Average(f1_list),Average(accuracy_list), Average(auroc_list), Average(auprc_list)))



---Fold1/5---
Classification Report:
               precision    recall  f1-score   support

           0       0.50      1.00      0.67     72765
           1       0.95      0.03      0.06     74088

    accuracy                           0.51    146853
   macro avg       0.73      0.51      0.36    146853
weighted avg       0.73      0.51      0.36    146853

F1 score: 0.05565746030082281
Accuracy: 0.509196271101033
AUPRC: 0.6857683848643801
AUROC: 0.7032685228089921
AUROC threshold: [1.89973961e+00 8.99739614e-01 8.94307322e-01 ... 4.04436729e-05
 3.42567985e-05 0.00000000e+00]
---Fold2/5---
Classification Report:
               precision    recall  f1-score   support

           0       0.51      1.00      0.67     74649
           1       0.90      0.02      0.04     74088

    accuracy                           0.51    148737
   macro avg       0.70      0.51      0.35    148737
weighted avg       0.70      0.51      0.36    148737

F1 score: 0.036805436205232606
Accuracy: 0.510

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

### Baseline Data W Feature Selection

In [20]:
f1_list = []
accuracy_list = []
auprc_list = []
auroc_list = []

for k in range(5):
    print('---Fold{}/5---'.format(k+1))
    
    # load the train and val data
    filename = '../data/data_both/kFold_baseline/fold{}/train.pickle'.format(k+1)
    with open(filename, 'rb') as f:
        train_baseline = pickle.load(f)

    filename = '../data/data_both/kFold_baseline/fold{}/val.pickle'.format(k+1)
    with open(filename, 'rb') as f:
        val_baseline = pickle.load(f)
    
    X_train_baseline = train_baseline[X_feature_baseline_fs]
    y_train_baseline = train_baseline[y_feature]

    X_val_baseline = val_baseline[X_feature_baseline_fs]
    y_val_baseline = val_baseline[y_feature]

    # train and save the models
    best_RF = RandomForestClassifier(random_state=1211, oob_score=True, max_depth=30, n_estimators=150, class_weight='balanced_subsample')
    best_RF.fit(X_train_baseline,y_train_baseline.values.ravel())
    joblib.dump(best_RF, './trained_models/BDWFS/model{}.joblib'.format(k+1))

    # train results
    y_pred_class = best_RF.predict(X_val_baseline)
    y_pred_proba = best_RF.predict_proba(X_val_baseline)[::,1]
    precision_, recall_, thresholds_ = precision_recall_curve(y_val_baseline, y_pred_proba)
    auprc_ = auc(recall_, precision_)
    fpr_, tpr_, thresholds_ = roc_curve(y_val_baseline, y_pred_proba)
    auroc_ = auc(fpr_, tpr_)
    f1_ = f1_score(y_val_baseline,y_pred_class)
    accuracy_ = accuracy_score(y_val_baseline,y_pred_class)

    print('Classification Report:\n', classification_report(y_val_baseline, y_pred_class))
    print('F1 score:', f1_)
    print('Accuracy:', accuracy_)
    print('AUPRC:', auprc_)
    print('AUROC:', auroc_)
    print('AUROC threshold:', thresholds_)

    f1_list.append(f1_)
    accuracy_list.append(accuracy_)
    auprc_list.append(auprc_)
    auroc_list.append(auroc_)

    # Plot ROC,PRC and confusion matrix
    bc = BinaryClassification(y_val_baseline,y_pred_proba, labels=['Non-sepsis', 'Sepsis'])
    plt.figure(figsize=(20,20))
    plt.subplot2grid(shape=(2,4), loc=(0,0), colspan=2)
    bc.plot_roc_curve()
    plt.subplot2grid((2,4), (0,2), colspan=2)
    bc.plot_precision_recall_curve()
    plt.subplot2grid((2,4), (1,0), colspan=2)
    bc.plot_confusion_matrix()
    plt.subplot2grid((2,4), (1,2), colspan=2)
    bc.plot_confusion_matrix(normalize=True)
    plt.savefig('./figs/val_w_fs_baseline_{}.pdf'.format(k+1))
    plt.clf()

def Average(lst):
    return sum(lst) / len(lst)

print('Averaged trauining results:\n F1_score:{}\n Accuracy:{}\n AUROC:{}\n AUPRC:{}\n'.format(Average(f1_list),Average(accuracy_list), Average(auroc_list), Average(auprc_list)))



---Fold1/5---
Classification Report:
               precision    recall  f1-score   support

           0       0.50      1.00      0.67     72765
           1       0.89      0.03      0.06     74088

    accuracy                           0.51    146853
   macro avg       0.70      0.51      0.36    146853
weighted avg       0.70      0.51      0.36    146853

F1 score: 0.05828589335143887
Accuracy: 0.5088694136313184
AUPRC: 0.6752901379196954
AUROC: 0.6920908620570799
AUROC threshold: [1.91313208e+00 9.13132079e-01 9.13083766e-01 ... 1.85278067e-04
 8.57950208e-05 0.00000000e+00]
---Fold2/5---
Classification Report:
               precision    recall  f1-score   support

           0       0.51      1.00      0.67     74649
           1       0.85      0.02      0.04     74088

    accuracy                           0.51    148737
   macro avg       0.68      0.51      0.35    148737
weighted avg       0.68      0.51      0.36    148737

F1 score: 0.03862745874064986
Accuracy: 0.510

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

### Engineered Data W/o Feature Selection

In [21]:
f1_list = []
accuracy_list = []
auprc_list = []
auroc_list = []

for k in range(5):
    print('---Fold{}/5---'.format(k+1))
    
    # load the train and val data
    filename = '../data/data_both/kFold_engineered/fold{}/train.pickle'.format(k+1)
    with open(filename, 'rb') as f:
        train_engineered = pickle.load(f)

    filename = '../data/data_both/kFold_engineered/fold{}/val.pickle'.format(k+1)
    with open(filename, 'rb') as f:
        val_engineered = pickle.load(f)
    
    X_train_engineered = train_engineered[X_feature_engineered]
    y_train_engineered = train_engineered[y_feature]

    X_val_engineered = val_engineered[X_feature_engineered]
    y_val_engineered = val_engineered[y_feature]

    # train and save the models
    best_RF = RandomForestClassifier(random_state=1211, oob_score=True, max_depth=30, n_estimators=50, class_weight='balanced_subsample')
    best_RF.fit(X_train_engineered,y_train_engineered.values.ravel())
    joblib.dump(best_RF, './trained_models/EDWOFS/model{}.joblib'.format(k+1))

    # train results
    y_pred_class = best_RF.predict(X_val_engineered)
    y_pred_proba = best_RF.predict_proba(X_val_engineered)[::,1]
    precision_, recall_, thresholds_ = precision_recall_curve(y_val_engineered, y_pred_proba)
    auprc_ = auc(recall_, precision_)
    fpr_, tpr_, thresholds_ = roc_curve(y_val_engineered, y_pred_proba)
    auroc_ = auc(fpr_, tpr_)
    f1_ = f1_score(y_val_engineered,y_pred_class)
    accuracy_ = accuracy_score(y_val_engineered,y_pred_class)

    print('Classification Report:\n', classification_report(y_val_engineered, y_pred_class))
    print('F1 score:', f1_)
    print('Accuracy:', accuracy_)
    print('AUPRC:', auprc_)
    print('AUROC:', auroc_)
    print('AUROC threshold:', thresholds_)

    f1_list.append(f1_)
    accuracy_list.append(accuracy_)
    auprc_list.append(auprc_)
    auroc_list.append(auroc_)

    # Plot ROC,PRC and confusion matrix
    bc = BinaryClassification(y_val_engineered,y_pred_proba, labels=['Non-sepsis', 'Sepsis'])
    plt.figure(figsize=(20,20))
    plt.subplot2grid(shape=(2,4), loc=(0,0), colspan=2)
    bc.plot_roc_curve()
    plt.subplot2grid((2,4), (0,2), colspan=2)
    bc.plot_precision_recall_curve()
    plt.subplot2grid((2,4), (1,0), colspan=2)
    bc.plot_confusion_matrix()
    plt.subplot2grid((2,4), (1,2), colspan=2)
    bc.plot_confusion_matrix(normalize=True)
    plt.savefig('./figs/val_wo_fs_engineered_{}.pdf'.format(k+1))
    plt.clf()

def Average(lst):
    return sum(lst) / len(lst)

print('Averaged trauining results:\n F1_score:{}\n Accuracy:{}\n AUROC:{}\n AUPRC:{}\n'.format(Average(f1_list),Average(accuracy_list), Average(auroc_list), Average(auprc_list)))



---Fold1/5---
Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.98      0.74     72765
           1       0.95      0.33      0.48     74088

    accuracy                           0.65    146853
   macro avg       0.77      0.65      0.61    146853
weighted avg       0.77      0.65      0.61    146853

F1 score: 0.4841540270783704
Accuracy: 0.6505280790995077
AUPRC: 0.8859369513348421
AUROC: 0.8851074424724292
AUROC threshold: [2.00000000e+00 1.00000000e+00 9.99875730e-01 ... 5.26538605e-05
 5.24229965e-05 0.00000000e+00]
---Fold2/5---
Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.98      0.75     74649
           1       0.95      0.35      0.51     74088

    accuracy                           0.66    148737
   macro avg       0.78      0.66      0.63    148737
weighted avg       0.77      0.66      0.63    148737

F1 score: 0.5067529460654813
Accuracy: 0.66483

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

### Engineered Data W Feature Selection

In [24]:
f1_list = []
accuracy_list = []
auprc_list = []
auroc_list = []

for k in range(5):
    print('---Fold{}/5---'.format(k+1))
    
    # load the train and val data
    filename = '../data/data_both/kFold_engineered/fold{}/train.pickle'.format(k+1)
    with open(filename, 'rb') as f:
        train_engineered = pickle.load(f)

    filename = '../data/data_both/kFold_engineered/fold{}/val.pickle'.format(k+1)
    with open(filename, 'rb') as f:
        val_engineered = pickle.load(f)
    
    X_train_engineered = train_engineered[X_feature_engineered_fs]
    y_train_engineered = train_engineered[y_feature]

    X_val_engineered = val_engineered[X_feature_engineered_fs]
    y_val_engineered = val_engineered[y_feature]

    # train and save the models
    best_RF = RandomForestClassifier(random_state=1211, oob_score=True, max_depth=30, n_estimators=150, class_weight='balanced_subsample')
    best_RF.fit(X_train_engineered,y_train_engineered.values.ravel())
    joblib.dump(best_RF, './trained_models/EDWFS/model{}.joblib'.format(k+1))

    # train results
    y_pred_class = best_RF.predict(X_val_engineered)
    y_pred_proba = best_RF.predict_proba(X_val_engineered)[::,1]
    precision_, recall_, thresholds_ = precision_recall_curve(y_val_engineered, y_pred_proba)
    auprc_ = auc(recall_, precision_)
    fpr_, tpr_, thresholds_ = roc_curve(y_val_engineered, y_pred_proba)
    auroc_ = auc(fpr_, tpr_)
    f1_ = f1_score(y_val_engineered,y_pred_class)
    accuracy_ = accuracy_score(y_val_engineered,y_pred_class)

    print('Classification Report:\n', classification_report(y_val_engineered, y_pred_class))
    print('F1 score:', f1_)
    print('Accuracy:', accuracy_)
    print('AUPRC:', auprc_)
    print('AUROC:', auroc_)
    print('AUROC threshold:', thresholds_)

    f1_list.append(f1_)
    accuracy_list.append(accuracy_)
    auprc_list.append(auprc_)
    auroc_list.append(auroc_)

    # Plot ROC,PRC and confusion matrix
    bc = BinaryClassification(y_val_engineered,y_pred_proba, labels=['Non-sepsis', 'Sepsis'])
    plt.figure(figsize=(20,20))
    plt.subplot2grid(shape=(2,4), loc=(0,0), colspan=2)
    bc.plot_roc_curve()
    plt.subplot2grid((2,4), (0,2), colspan=2)
    bc.plot_precision_recall_curve()
    plt.subplot2grid((2,4), (1,0), colspan=2)
    bc.plot_confusion_matrix()
    plt.subplot2grid((2,4), (1,2), colspan=2)
    bc.plot_confusion_matrix(normalize=True)
    plt.savefig('./figs/val_w_fs_engineered_{}.pdf'.format(k+1))
    plt.clf()

def Average(lst):
    return sum(lst) / len(lst)

print('Averaged trauining results:\n F1_score:{}\n Accuracy:{}\n AUROC:{}\n AUPRC:{}\n'.format(Average(f1_list),Average(accuracy_list), Average(auroc_list), Average(auprc_list)))



---Fold1/5---
Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.98      0.74     72765
           1       0.95      0.35      0.51     74088

    accuracy                           0.66    146853
   macro avg       0.77      0.67      0.63    146853
weighted avg       0.78      0.66      0.63    146853

F1 score: 0.5107391029690461
Accuracy: 0.6624651862747101
AUPRC: 0.8893322446763836
AUROC: 0.890635229593534
AUROC threshold: [1.97324983e+00 9.73249830e-01 9.46608317e-01 ... 1.51892991e-05
 1.29993125e-05 0.00000000e+00]
---Fold2/5---
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.98      0.75     74649
           1       0.95      0.36      0.52     74088

    accuracy                           0.67    148737
   macro avg       0.78      0.67      0.64    148737
weighted avg       0.78      0.67      0.64    148737

F1 score: 0.5222234164167
Accuracy: 0.671238494

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

# Performance on Test data
 Four models using baseline data/ engineered data and with/ without feature engineering will be run on the test data

### Data Processing Functions

In [38]:
def save_challenge_predictions(file, scores, labels):
    with open(file, 'w') as f:
        f.write('PredictedProbability,PredictedLabel\n')
        for (s, l) in zip(scores, labels):
            f.write('%g,%d\n' % (s, l))

def save_challenge_testlabel(file, labels):
    with open(file, 'w') as f:
        f.write('sepsis\n')
        for l in labels:
            f.write('%d\n' % l)

def load_model_predict(X_test, k_fold, path):
    "ensemble the five XGBoost models by averaging their output probabilities"
    test_pred = np.zeros((X_test.shape[0], k_fold))
    for k in range(k_fold):
        # load the model
        model_path_name = path + 'model{}.joblib'.format(k+1) 
        loaded_model = joblib.load(model_path_name)
        # predict
        y_test_pred = loaded_model.predict_proba(X_test)[::,1]
        test_pred[:, k] = y_test_pred # save prediction results 5 times
    test_pred = pd.DataFrame(test_pred)
    result_pro = test_pred.mean(axis=1)

    return result_pro

def feature_extraction(case, data_features):
    labels = np.array(case['sepsis'])
    features = case[data_features]
    if 'time' in features.columns:
        features = features.drop(columns=['time'],axis = 1)

    return  features, labels    

def predict(data_set,
            data_dir,
            save_prediction_dir,
            save_label_dir,
            model_path,
            risk_threshold,
            data_features
            ):
    for csv in tqdm(data_set):
        csv = csv.replace('psv','csv')
        patient = pd.read_csv(data_dir+csv, sep=',')
        features, labels = feature_extraction(patient, data_features)

        predict_pro = load_model_predict(features, k_fold = 5, path = model_path)
        PredictedProbability = np.array(predict_pro)
        PredictedLabel = [0 if i <= risk_threshold else 1 for i in predict_pro]

        save_prediction_name = save_prediction_dir + csv
        save_challenge_predictions(save_prediction_name, PredictedProbability, PredictedLabel)
        save_testlabel_name = save_label_dir + csv
        save_challenge_testlabel(save_testlabel_name, labels)

###  Score Function

In [39]:
# This file contains functions for evaluating algorithms for the 2019 PhysioNet/
# CinC Challenge. You can run it as follows:

################################################################################

# The evaluate_scores function computes a normalized utility score for a cohort
# of patients along with several traditional scoring metrics.
#
# Inputs:
#   'label_directory' is a directory of pipe-delimited text files containing a
#   binary vector of labels for whether a patient is not septic (0) or septic
#   (1) for each time interval.
#
#   'prediction_directory' is a directory of pipe-delimited text files, where
#   the first column of the file gives the predicted probability that the
#   patient is septic at each time, and the second column of the file is a
#   binarized version of this vector. Note that there must be a prediction for
#   every label.
#
# Outputs:
#   'auroc' is the area under the receiver operating characteristic curve
#   (AUROC).
#
#   'auprc' is the area under the precision recall curve (AUPRC).
#
#   'accuracy' is accuracy.
#
#   'f_measure' is F-measure.
#
#   'normalized_observed_utility' is a normalized utility-based measure that we
#   created for the Challenge. This score is normalized so that a perfect score
#   is 1 and no positive predictions is 0.
#
# Example:
#   Omitted due to length. See the below examples.

import numpy as np, os, os.path, sys, warnings

def evaluate_sepsis_score(label_directory, prediction_directory):
    # Set parameters.
    label_header       = 'sepsis'
    prediction_header  = 'PredictedLabel'
    probability_header = 'PredictedProbability'

    dt_early   = -12
    dt_optimal = -6
    dt_late    = 3

    max_u_tp = 1
    min_u_fn = -2
    u_fp     = -0.05
    u_tn     = 0

    # Find label and prediction files.
    label_files = []
    for f in os.listdir(label_directory):
        g = os.path.join(label_directory, f)
        if os.path.isfile(g) and not f.lower().startswith('.') and f.lower().endswith('csv'):
            label_files.append(g)
    label_files = sorted(label_files)

    prediction_files = []
    for f in os.listdir(prediction_directory):
        g = os.path.join(prediction_directory, f)
        if os.path.isfile(g) and not f.lower().startswith('.') and f.lower().endswith('csv'):
            prediction_files.append(g)
    prediction_files = sorted(prediction_files)

    if len(label_files) != len(prediction_files):
        raise Exception('Numbers of label and prediction files must be the same.')

    # Load labels and predictions.
    num_files            = len(label_files)
    cohort_labels        = []
    cohort_predictions   = []
    cohort_probabilities = []

    for k in range(num_files):
        labels        = load_column(label_files[k], label_header, ',')
        predictions   = load_column(prediction_files[k], prediction_header, ',')
        probabilities = load_column(prediction_files[k], probability_header, ',')

        # Check labels and predictions for errors.
        if not (len(labels) == len(predictions) and len(predictions) == len(probabilities)):
            raise Exception('Numbers of labels and predictions for a file must be the same.')

        num_rows = len(labels)

        for i in range(num_rows):
            if labels[i] not in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

            if predictions[i] not in (0, 1):
                raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

            if not 0 <= probabilities[i] <= 1:
                warnings.warn('Probabilities do not satisfy 0 <= probability <= 1.')

        if 0 < np.sum(predictions) < num_rows:
            min_probability_positive = np.min(probabilities[predictions == 1])
            max_probability_negative = np.max(probabilities[predictions == 0])

            if min_probability_positive <= max_probability_negative:
                warnings.warn('Predictions are inconsistent with probabilities, i.e., a positive prediction has a lower (or equal) probability than a negative prediction.')

        # Record labels and predictions.
        cohort_labels.append(labels)
        cohort_predictions.append(predictions)
        cohort_probabilities.append(probabilities)

    # Compute AUC, accuracy, and F-measure.
    labels        = np.concatenate(cohort_labels)
    predictions   = np.concatenate(cohort_predictions)
    probabilities = np.concatenate(cohort_probabilities)

    auroc, auprc        = compute_auc(labels, probabilities)
    accuracy, f_measure = compute_accuracy_f_measure(labels, predictions)

    # Compute utility.
    observed_utilities = np.zeros(num_files)
    best_utilities     = np.zeros(num_files)
    worst_utilities    = np.zeros(num_files)
    inaction_utilities = np.zeros(num_files)

    for k in range(num_files):
        labels = cohort_labels[k]
        num_rows          = len(labels)
        observed_predictions = cohort_predictions[k]
        best_predictions     = np.zeros(num_rows)
        worst_predictions    = np.zeros(num_rows)
        inaction_predictions = np.zeros(num_rows)

        if np.any(labels):
            t_sepsis = np.argmax(labels) - dt_optimal
            best_predictions[max(0, t_sepsis + dt_early) : min(t_sepsis + dt_late + 1, num_rows)] = 1
        worst_predictions = 1 - best_predictions

        observed_utilities[k] = compute_prediction_utility(labels, observed_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        best_utilities[k]     = compute_prediction_utility(labels, best_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        worst_utilities[k]    = compute_prediction_utility(labels, worst_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        inaction_utilities[k] = compute_prediction_utility(labels, inaction_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)

    unnormalized_observed_utility = np.sum(observed_utilities)
    unnormalized__utility     = np.sum(best_utilities)
    unnormalized_worst_utility    = np.sum(worst_utilities)
    unnormalized_inaction_utility = np.sum(inaction_utilities)

    normalized_observed_utility = (unnormalized_observed_utility - unnormalized_inaction_utility) / (unnormalized__utility - unnormalized_inaction_utility)

    return auroc, auprc, accuracy, f_measure, normalized_observed_utility

# The load_column function loads a column from a table.
#
# Inputs:
#   'filename' is a string containing a filename.
#
#   'header' is a string containing a header.
#
# Outputs:
#   'column' is a vector containing a column from the file with the given
#   header.
#
# Example:
#   Omitted.

def load_column(filename, header, delimiter):
    column = []
    with open(filename, 'r') as f:
        for i, l in enumerate(f):
            arrs = l.strip().split(delimiter)
            if i == 0:
                try:
                    j = arrs.index(header)
                except:
                    raise Exception('{} must contain column with header {} containing numerical entries.'.format(filename, header))
            else:
                if len(arrs[j]):
                    column.append(float(arrs[j]))
    return np.array(column)

# The compute_auc function computes AUROC and AUPRC as well as other summary
# statistics (TP, FP, FN, TN, tpr_, TNR, PPV, NPV, etc.) that can be exposed
# from this function.
#
# Inputs:
#   'labels' is a binary vector, where labels[i] == 0 if the patient is not
#   labeled as septic at time i and labels[i] == 1 if the patient is labeled as
#   septic at time i.
#
#   'predictions' is a probability vector, where predictions[i] gives the
#   predicted probability that the patient is septic at time i.  Note that there
#   must be a prediction for every label, i.e, len(labels) ==
#   len(predictions).
#
# Outputs:
#   'auroc' is a scalar that gives the AUROC of the algorithm using its
#   predicted probabilities, where specificity is interpolated for intermediate
#   sensitivity values.
#
#   'auprc' is a scalar that gives the AUPRC of the algorithm using its
#   predicted probabilities, where precision is a piecewise constant function of
#   recall.
#
# Example:
#   In [1]: labels = [0, 0, 0, 0, 1, 1]
#   In [2]: predictions = [0.3, 0.4, 0.6, 0.7, 0.8, 0.8]
#   In [3]: auroc, auprc = compute_auc(labels, predictions)
#   In [4]: auroc
#   Out[4]: 1.0
#   In [5]: auprc
#   Out[5]: 1.0

def compute_auc(labels, predictions, check_errors=True):
    # Check inputs for errors.
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not 0 <= prediction <= 1:
                warnings.warn('Predictions do not satisfy 0 <= prediction <= 1.')

    # Find prediction thresholds_.
    thresholds_ = np.unique(predictions)[::-1]
    if thresholds_[0] != 1:
        thresholds_ = np.insert(thresholds_, 0, 1)
    if thresholds_[-1] == 0:
        thresholds_ = thresholds_[:-1]

    n = len(labels)
    m = len(thresholds_)

    # Populate contingency table across prediction thresholds_.
    tp = np.zeros(m)
    fp = np.zeros(m)
    fn = np.zeros(m)
    tn = np.zeros(m)

    # Find indices that sort the predicted probabilities from largest to
    # smallest.
    idx = np.argsort(predictions)[::-1]

    i = 0
    for j in range(m):
        # Initialize contingency table for j-th prediction threshold.
        if j == 0:
            tp[j] = 0
            fp[j] = 0
            fn[j] = np.sum(labels)
            tn[j] = n - fn[j]
        else:
            tp[j] = tp[j - 1]
            fp[j] = fp[j - 1]
            fn[j] = fn[j - 1]
            tn[j] = tn[j - 1]

        # Update contingency table for i-th largest predicted probability.
        while i < n and predictions[idx[i]] >= thresholds_[j]:
            if labels[idx[i]]:
                tp[j] += 1
                fn[j] -= 1
            else:
                fp[j] += 1
                tn[j] -= 1
            i += 1

    # Summarize contingency table.
    tpr_ = np.zeros(m)
    tnr = np.zeros(m)
    ppv = np.zeros(m)
    npv = np.zeros(m)

    for j in range(m):
        if tp[j] + fn[j]:
            tpr_[j] = tp[j] / (tp[j] + fn[j])
        else:
            tpr_[j] = 1
        if fp[j] + tn[j]:
            tnr[j] = tn[j] / (fp[j] + tn[j])
        else:
            tnr[j] = 1
        if tp[j] + fp[j]:
            ppv[j] = tp[j] / (tp[j] + fp[j])
        else:
            ppv[j] = 1
        if fn[j] + tn[j]:
            npv[j] = tn[j] / (fn[j] + tn[j])
        else:
            npv[j] = 1

    # Compute AUROC as the area under a piecewise linear function with tpr_ /
    # sensitivity (x-axis) and TNR / specificity (y-axis) and AUPRC as the area
    # under a piecewise constant with tpr_ / recall (x-axis) and PPV / precision
    # (y-axis).
    auroc = 0
    auprc = 0
    for j in range(m-1):
        auroc += 0.5 * (tpr_[j + 1] - tpr_[j]) * (tnr[j + 1] + tnr[j])
        auprc += (tpr_[j + 1] - tpr_[j]) * ppv[j + 1]

    return auroc, auprc

# The compute_accuracy_f_measure function computes the accuracy and F-measure
# for a patient.
#
# Inputs:
#   'labels' is a binary vector, where labels[i] == 0 if the patient is not
#   labeled as septic at time i and labels[i] == 1 if the patient is labeled as
#   septic at time i.
#
#   'predictions' is a binary vector, where predictions[i] == 0 if the patient
#   is not predicted to be septic at time i and predictions[i] == 1 if the
#   patient is predicted to be septic at time i.  Note that there must be a
#   prediction for every label, i.e, len(labels) == len(predictions).
#
# Output:
#   'accuracy' is a scalar that gives the accuracy of the predictions using its
#   binarized predictions.
#
#   'f_measure' is a scalar that gives the F-measure of the predictions using its
#   binarized predictions.
#
# Example:
#   In [1]: labels = [0, 0, 0, 0, 1, 1]
#   In [2]: predictions = [0, 0, 1, 1, 1, 1]
#   In [3]: accuracy, f_measure = compute_accuracy_f_measure(labels, predictions)
#   In [4]: accuracy
#   Out[4]: 0.666666666667
#   In [5]: f_measure
#   Out[5]: 0.666666666667

def compute_accuracy_f_measure(labels, predictions, check_errors=True):
    # Check inputs for errors.
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not prediction in (0, 1):
                raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

    # Populate contingency table.
    n = len(labels)
    tp = 0
    fp = 0
    fn = 0
    tn = 0

    for i in range(n):
        if labels[i] and predictions[i]:
            tp += 1
        elif not labels[i] and predictions[i]:
            fp += 1
        elif labels[i] and not predictions[i]:
            fn += 1
        elif not labels[i] and not predictions[i]:
            tn += 1

    # Summarize contingency table.
    if tp + fp + fn + tn:
        accuracy = float(tp + tn) / float(tp + fp + fn + tn)
    else:
        accuracy = 1.0

    if 2 * tp + fp + fn:
        f_measure = float(2 * tp) / float(2 * tp + fp + fn)
    else:
        f_measure = 1.0

    return accuracy, f_measure

# The compute_prediction_utility function computes the total time-dependent
# utility for a patient.
#
# Inputs:
#   'labels' is a binary vector, where labels[i] == 0 if the patient is not
#   labeled as septic at time i and labels[i] == 1 if the patient is labeled as
#   septic at time i.
#
#   'predictions' is a binary vector, where predictions[i] == 0 if the patient
#   is not predicted to be septic at time i and predictions[i] == 1 if the
#   patient is predicted to be septic at time i.  Note that there must be a
#   prediction for every label, i.e, len(labels) == len(predictions).
#
# Output:
#   'utility' is a scalar that gives the total time-dependent utility of the
#   algorithm using its binarized predictions.
#
# Example:
#   In [1]: labels = [0, 0, 0, 0, 1, 1]
#   In [2]: predictions = [0, 0, 1, 1, 1, 1]
#   In [3]: utility = compute_prediction_utility(labels, predictions)
#   In [4]: utility
#   Out[4]: 3.388888888888889

def compute_prediction_utility(labels, predictions, dt_early=-12, dt_optimal=-6, dt_late=3.0, max_u_tp=1, min_u_fn=-2, u_fp=-0.05, u_tn=0, check_errors=True):
    # Check inputs for errors.
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not prediction in (0, 1):
                raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

        if dt_early >= dt_optimal:
            raise Exception('The earliest beneficial time for predictions must be before the optimal time.')

        if dt_optimal >= dt_late:
            raise Exception('The optimal time for predictions must be before the latest beneficial time.')

    # Does the patient eventually have sepsis?
    if np.any(labels):
        is_septic = True
        t_sepsis = np.argmax(labels) - dt_optimal
    else:
        is_septic = False
        t_sepsis = float('inf')

    n = len(labels)

    # Define slopes and intercept points for utility functions of the form
    # u = m * t + b.
    m_1 = float(max_u_tp) / float(dt_optimal - dt_early)
    b_1 = -m_1 * dt_early
    m_2 = float(-max_u_tp) / float(dt_late - dt_optimal)
    b_2 = -m_2 * dt_late
    m_3 = float(min_u_fn) / float(dt_late - dt_optimal)
    b_3 = -m_3 * dt_optimal

    # Compare predicted and true conditions.
    u = np.zeros(n)
    for t in range(n):
        if t <= t_sepsis + dt_late:
            # TP
            if is_septic and predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = max(m_1 * (t - t_sepsis) + b_1, u_fp)
                elif t <= t_sepsis + dt_late:
                    u[t] = m_2 * (t - t_sepsis) + b_2
            # FP
            elif not is_septic and predictions[t]:
                u[t] = u_fp
            # FN
            elif is_septic and not predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = 0
                elif t <= t_sepsis + dt_late:
                    u[t] = m_3 * (t - t_sepsis) + b_3
            # TN
            elif not is_septic and not predictions[t]:
                u[t] = u_tn

    # Find total utility for patient.
    return np.sum(u)

### Baseline Data W/o Feature Selection

In [78]:
!mkdir -p ./prediction
!mkdir -p ./label

In [39]:
test_set = np.load('../data/data_both/test_set.npy')
test_data_path = '../data/data_both/test_baseline/' #change here
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/BDWOFS/' #change here

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_baseline) # TODO: adjust the threshold

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 1446/1446 [3:16:33<00:00,  8.16s/it]  


AUROC|AUPRC|Accuracy|F-measure|Utility
0.7158186118459031|0.19171333761576162|0.921391216684519|0.05997588908981314|0.028938887943503768


Results:

AUROC|AUPRC|Accuracy|F-measure|Utility

0.7158186118459031|0.19171333761576162|0.921391216684519|0.05997588908981314|0.028938887943503768

### Baseline Data W Feature Selection

In [40]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [41]:
test_set = np.load('../data/data_both/test_set.npy')
test_data_path = '../data/data_both/test_baseline/' #change here
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/BDWFS/' #change here

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_baseline_fs) # TODO: adjust the threshold

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 1446/1446 [2:39:11<00:00,  6.61s/it] 


AUROC|AUPRC|Accuracy|F-measure|Utility
0.6956125093619776|0.17322611162308926|0.9226891815260538|0.0657834627683874|0.032075852973708796


Results:

AUROC|AUPRC|Accuracy|F-measure|Utility

0.6956125093619776|0.17322611162308926|0.9226891815260538|0.0657834627683874|0.032075852973708796

### Engineered Data W/o Feature Selection

In [42]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [44]:
test_set = np.load('../data/data_both/test_set.npy')
test_data_path = '../data/data_both/test_engineered/' #change here
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/EDWOFS/'

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_engineered) # TODO: adjust the threshold

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 1446/1446 [30:25<00:00,  1.26s/it]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.896089699263837|0.5248505675671032|0.9330602986579296|0.4485049833887043|0.3317807155619842


Results:

AUROC|AUPRC|Accuracy|F-measure|Utility

0.896089699263837|0.5248505675671032|0.9330602986579296|0.4485049833887043|0.3317807155619842

### Engineered Data W Feature Selection

In [45]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [46]:
test_set = np.load('../data/data_both/test_set.npy')
test_data_path = '../data/data_both/test_engineered/' #change here
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/EDWFS/'

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_engineered_fs) # TODO: adjust the threshold

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 1446/1446 [1:32:59<00:00,  3.86s/it]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.8967426270456575|0.5192059048392439|0.933035095457123|0.4473793677204659|0.33049061941631924


Results:

AUROC|AUPRC|Accuracy|F-measure|Utility

0.8967426270456575|0.5192059048392439|0.933035095457123|0.4473793677204659|0.33049061941631924

## Test Real Neonatal data

### BDWOFS

In [22]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [23]:
# load test data
test_set = np.load('../../models_neoantes/data/test_set_balanced.npy')
test_data_path = '../../datasets/MIMICIII/neonates/baseline_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/BDWOFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_baseline)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [25:43<00:00,  6.83s/it]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.5163792910224353|0.03972504224365393|0.957329435616955|0.0|-0.004262744609248446


### BDWFS

In [24]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [25]:
# load test data
test_set = np.load('../../models_neoantes/data/test_set_balanced.npy')
test_data_path = '../../datasets/MIMICIII/neonates/baseline_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/BDWFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_baseline_fs)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [29:11<00:00,  7.75s/it]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.48368631010118573|0.03716245806552646|0.7682587580744024|0.05243878928089454|0.02599193983962768


### EDWOFS

In [26]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [27]:
# load test data
test_set = np.load('../../models_neoantes/data/test_set_balanced.npy')
test_data_path = '../../datasets/MIMICIII/neonates/engineered_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/EDWOFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_engineered)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [06:12<00:00,  1.65s/it]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.7447344494997661|0.1568328835086242|0.9627045122353718|0.0|0.0


### EDWFS

In [28]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [29]:
# load test data
test_set = np.load('../../models_neoantes/data/test_set_balanced.npy')
test_data_path = '../../datasets/MIMICIII/neonates/engineered_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/EDWFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_engineered_fs)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [19:15<00:00,  5.11s/it]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.7740359544929439|0.2589391903989999|0.9629874110047622|0.017521902377972465|0.008517179774814011


## Test Artificial Neonatal Data
* ### BDWOFS

In [40]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [41]:
# load test data
test_set = np.load('../../artificial_neonatal_data/data/balanced_226/test_set_balanced.npy')
test_data_path = '../../artificial_neonatal_data/data/balanced_226/baseline_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/BDWOFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_baseline)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [27:55<00:00,  7.41s/it]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.613897421512719|0.07106831976229978|0.9610828274916781|0.002509410288582183|0.0005608874485853173


* ### BDWFS

In [42]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [43]:
# load test data
test_set = np.load('../../artificial_neonatal_data/data/balanced_226/test_set_balanced.npy')
test_data_path = '../../artificial_neonatal_data/data/balanced_226/baseline_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/BDWFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_baseline_fs)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [31:29<00:00,  8.36s/it]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.6232119286156251|0.07000360316373797|0.960544350890934|0.0024752475247524753|0.0003116041381029257


* ### EDWOFS

In [44]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [45]:
# load test data
test_set = np.load('../../artificial_neonatal_data/data/balanced_226/test_set_balanced.npy')
test_data_path = '../../artificial_neonatal_data/data/balanced_226/engineered_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/EDWOFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_engineered)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [06:43<00:00,  1.78s/it]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.8096699083305097|0.35022191608695963|0.961278637164676|0.0|0.0


* ### EDWFS

In [46]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [47]:
# load test data
test_set = np.load('../../artificial_neonatal_data/data/balanced_226/test_set_balanced.npy')
test_data_path = '../../artificial_neonatal_data/data/balanced_226/engineered_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/EDWFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_engineered_fs)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [20:51<00:00,  5.54s/it]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.8205268544435486|0.4251502012940246|0.9615233992559232|0.015037593984962405|0.006439818854127698
